In [1]:
"""
Copyright (C) 2018 NVIDIA Corporation.  All rights reserved.
Licensed under the CC BY-NC-SA 4.0 license (https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode).
"""
"""
Copyright (C) 2018 NVIDIA Corporation.  All rights reserved.
Licensed under the CC BY-NC-SA 4.0 license (https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode).
"""

'\nCopyright (C) 2018 NVIDIA Corporation.  All rights reserved.\nLicensed under the CC BY-NC-SA 4.0 license (https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode).\n'

In [2]:
!pip3 install torchvision
!pip3 install --no-cache-dir -I pillow

    100% |████████████████████████████████| 2.0MB 36.0MB/s 


In [3]:
!pip install cupy-cuda92


In [4]:
!pip install pynvrtc

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [6]:
%cd gdrive/
%cd My Drive
!ls

/content/gdrive
/content/gdrive/My Drive
 20181005-AdaptSeg--HINGE-2DA-PCONV-250k-acc   Gated-bce-ttur-1108
 20181022-AdaptSeg-2DA-Gated-SA-250k-acc4643   Gated-Change-Class-1123
 AdaptSegNet				       Gated-multi-1109
 Checkoutput				       Gated-stable-1119
'Colab Notebooks'			       GATED_STABLE_1121
 Demo-image.gslides			       Gated-stable-1v1-1117
 FastPhotoStyle				       Gated-TTUR-1109
 Gated-1v2-ch-fore-1126			       Gated-TTUR-250k-acc4596
 Gated-1v5-1121				       Gated-TTUR-linux-1108
 Gated-250k-continue-1115		       Gated-TTUR-linux-1109
 Gated-250k-continue-1117		       Hinge-feature
 Gated-250k-continue-1119		       PConv-TTUR-1v5-250k-acc4566-1023
 Gated-bce				       tirple-1119
 Gated-bce-1107				       transfer-test
 gated-bce-1108				       Triple-1117
 Gated-bce-TTUR				       triple-model
 GATED-BCE-ttur-1107			       Untitled0.ipynb


In [7]:
%cd FastPhotoStyle/

/content/gdrive/My Drive/FastPhotoStyle


In [0]:
from __future__ import print_function
import argparse
import torch
import process_stylization
from photo_wct import PhotoWCT
from photo_smooth import Propagator

In [0]:
from os import path

In [0]:
import os
import time

In [0]:
model ='./PhotoWCTModels/photo_wct.pth'
content_image_path ='./images/content1.png'
content_seg_path =[]
style_image_path ='./images/style1.png'
style_seg_path = []
output_image_path = './results/example1.png'
save_intermediate = False
fast = True
no_post = False
cuda = 0


# setup model

In [0]:
p_wct = PhotoWCT()

In [0]:
p_wct.load_state_dict(torch.load(model))

In [14]:
p_wct.cuda(0)

PhotoWCT(
  (e1): VGGEncoder(
    (conv0): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (pad1_1): ReflectionPad2d((1, 1, 1, 1))
    (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (relu1_1): ReLU(inplace)
  )
  (d1): VGGDecoder(
    (pad1_1): ReflectionPad2d((1, 1, 1, 1))
    (conv1_1): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1))
  )
  (e2): VGGEncoder(
    (conv0): Conv2d(3, 3, kernel_size=(1, 1), stride=(1, 1))
    (pad1_1): ReflectionPad2d((1, 1, 1, 1))
    (conv1_1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (relu1_1): ReLU(inplace)
    (pad1_2): ReflectionPad2d((1, 1, 1, 1))
    (conv1_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (relu1_2): ReLU(inplace)
    (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (pad2_1): ReflectionPad2d((1, 1, 1, 1))
    (conv2_1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (relu2_1): ReLU(inplace)
  )
  (d2): VGGDecoder(
    (pad2_1): ReflectionPa

# Load model

In [0]:
p_pro = Propagator()

#  Load CityScapes dataset

In [0]:
def get_Cityscapes_Dataset(root_path, list_path):
    img_ids = [i_id.strip() for i_id in open(list_path)]
    # for split in ["train", "trainval", "val"]:
#     split_set = "train"
    files = []
    for name in img_ids:
        img_file = path.join(root_path, "leftImg8bit/train/%s" % (name))
        files.append({
            "img": img_file,
            "name": str(name).replace("/","-")
        })
    print(files[:5])
    return files

In [17]:
data_directory =  path.join("..", "AdaptSegNet", "data", "Cityscapes", "data")
list_path = path.join("..", "AdaptSegNet", "dataset", "cityscapes_list", "train.txt")
CityScapes_dataset = get_Cityscapes_Dataset(data_directory, list_path)

[{'img': '../AdaptSegNet/data/Cityscapes/data/leftImg8bit/train/aachen/aachen_000000_000019_leftImg8bit.png', 'name': 'aachen-aachen_000000_000019_leftImg8bit.png'}, {'img': '../AdaptSegNet/data/Cityscapes/data/leftImg8bit/train/aachen/aachen_000001_000019_leftImg8bit.png', 'name': 'aachen-aachen_000001_000019_leftImg8bit.png'}, {'img': '../AdaptSegNet/data/Cityscapes/data/leftImg8bit/train/aachen/aachen_000002_000019_leftImg8bit.png', 'name': 'aachen-aachen_000002_000019_leftImg8bit.png'}, {'img': '../AdaptSegNet/data/Cityscapes/data/leftImg8bit/train/aachen/aachen_000003_000019_leftImg8bit.png', 'name': 'aachen-aachen_000003_000019_leftImg8bit.png'}, {'img': '../AdaptSegNet/data/Cityscapes/data/leftImg8bit/train/aachen/aachen_000004_000019_leftImg8bit.png', 'name': 'aachen-aachen_000004_000019_leftImg8bit.png'}]


In [0]:
import random

In [19]:
for _ in range(3):
    print(random.choice(CityScapes_dataset)['img'])

../AdaptSegNet/data/Cityscapes/data/leftImg8bit/train/cologne/cologne_000097_000019_leftImg8bit.png
../AdaptSegNet/data/Cityscapes/data/leftImg8bit/train/aachen/aachen_000095_000019_leftImg8bit.png
../AdaptSegNet/data/Cityscapes/data/leftImg8bit/train/tubingen/tubingen_000007_000019_leftImg8bit.png


# Load Synthia dataset

In [0]:
def get_SYNTHIA_Dataset(root_path, list_path):
    img_ids = [i_id.strip() for i_id in open(list_path)]
    # for split in ["train", "trainval", "val"]:
#     split_set = "train"
    files = []
    for name in img_ids:
        img_file = path.join(root_path, "%s" % (name))
        files.append({
            "img": img_file,
            "name": name
        })
    print(files[:5])
    return files

In [21]:
data_directory =  path.join("..", "AdaptSegNet", "data", "SYNTHIA")
list_path = path.join("..", "AdaptSegNet", "dataset", "synthia_list", "train.txt")

SYNTHIA_dataset = get_SYNTHIA_Dataset(data_directory, list_path)

[{'img': '../AdaptSegNet/data/SYNTHIA/0000000.png', 'name': '0000000.png'}, {'img': '../AdaptSegNet/data/SYNTHIA/0000001.png', 'name': '0000001.png'}, {'img': '../AdaptSegNet/data/SYNTHIA/0000002.png', 'name': '0000002.png'}, {'img': '../AdaptSegNet/data/SYNTHIA/0000003.png', 'name': '0000003.png'}, {'img': '../AdaptSegNet/data/SYNTHIA/0000004.png', 'name': '0000004.png'}]


In [22]:
output_image_root = path.join("..", "transfer-test")
if not os.path.exists(output_image_root):
  os.mkdir(output_image_root)
total = len(SYNTHIA_dataset)

for index, item in enumerate(SYNTHIA_dataset):
#     transfer SYNTHIA to cityscapes
#     for _ in range(3):
    t = time.time()
    style_image_path = random.choice(CityScapes_dataset)['img']
    image_path = item['img']
    output_image_path = path.join(output_image_root, item['name'])
#     print("output_image_path =", output_image_path)
    
    process_stylization.stylization(
        stylization_module=p_wct,
        smoothing_module=p_pro,
        content_image_path=image_path,
        style_image_path=style_image_path,
        content_seg_path=[],
        style_seg_path=[],
        output_image_path=output_image_path,
        cuda=1,
        save_intermediate=False,
        no_post=False
    )
    print("%d/%d : cost time = %s output_image_path = %s" % (index, total, time.time()-t, output_image_path))

Resize image: (1280,760)->(1280,760)
Resize image: (2048,1024)->(960,480)
Elapsed time in stylization: 2.094681
Elapsed time in propagation: 673.704083
Elapsed time in post processing: 0.032908


OSError: ignored